This notebook evaluates the results form the user study. Set `FILL_DATABASE` to True to create a mock database. Set it to False to use the results from an existing database.

If mocking data, this notebook expects the server to run on `server_url` with a database with no entries, as set up by running `node setup.mjs`. 

In [1]:
USER_STUDY_CSV = "./dataset_user_study.csv"
SQLITE_DB = "../survey/db.db"
FILL_DATABASE = True # if True, data is mocked, THIS CALLS THE APIs

server_url = "http://localhost:3002" # server to mock data on


In [2]:

import krippendorff
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp
import numpy as np 
import scipy.stats as stats 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Adapted from Hase et al. 2020. Main points:

This is a within-subject* design with 4 phases: (1) Predictions only, (2) Pre-learn test, (3) Teaching: Predictions + Explanations, (4) Eval.

Phase 1 and 3 share a set of documents as do 2 and 4.

Result Hase et al.: Report **average change** in user accuracy per explanation method (phase 2 vs. 4), CI and p values of mean

Additional details by Hase et al.:
- Balance data "by model correctness" so random guessing can't succeed: *"we ensure that true positives, false positives,
true negatives, and false negatives are equally represented in the inputs. [...] We confirm user understanding of the data
balancing in our screening test"*
- Forced choice, to not "favor overly niche explanations" (like in Ribeiro et al.)
- Separate teach and test phases
- Pre prediction phase to obtain a baseline



In [3]:
import sqlite3
import pandas as pd
connection = sqlite3.connect(SQLITE_DB)

user_df = pd.read_sql_query("SELECT * FROM users", connection)

In [4]:
user_df

,ID,access_token,current_phase,detector,explainer,document_order_a,document_order_b
0,1,YOEMSM,-1,None,None,"[12,11,1,6,16,15,2,9,13,8,0,4,14,7,17,5,3,10]","[15,14,0,10,9,7,13,8,3,5,16,2,4,12,1,6,17,11]"
1,2,LDJVYN,-1,None,None,"[11,4,15,1,10,14,2,13,5,8,7,16,6,0,9,3,17,12]","[2,10,6,1,5,4,15,12,16,8,3,17,9,11,7,14,0,13]"
2,3,DDYXJD,-1,None,None,"[6,4,8,9,5,14,0,13,15,12,10,16,7,1,3,2,17,11]","[10,15,17,1,0,11,9,2,13,7,16,14,3,4,5,12,6,8]"
3,4,KHGHUI,-1,None,None,"[8,16,6,4,9,7,17,1,12,0,2,13,11,15,10,5,3,14]","[9,15,6,3,13,14,16,12,2,17,4,1,7,8,10,0,5,11]"
4,5,LIQXER,-1,None,None,"[2,1,13,8,6,12,15,14,7,4,0,17,3,11,9,10,16,5]","[12,14,15,4,6,13,3,8,2,11,16,9,1,0,5,10,7,17]"
5,6,ZLWEGL,-1,None,None,"[10,2,1,3,0,5,15,17,16,13,11,9,14,4,12,6,7,8]","[14,17,6,1,8,12,5,13,11,3,9,7,16,4,0,10,2,15]"
6,7,YOBXBC,-1,None,None,"[12,3,17,14,11,2,9,7,8,4,1,16,6,5,15,10,0,13]","[8,15,3,13,4,10,6,17,5,0,12,11,1,9,14,7,2,16]"
7,8,LKKCKR,-1,None,None,"[5,15,3,11,1,0,6,2,10,12,16,17,14,4,13,9,8,7]","[2,9,16,15,1,6,3,17,7,11,0,13,4,8,10,5,12,14]"
8,9,FXPIKS,-1,None,None,"[5,8,9,0,15,2,14,17,16,1,4,12,7,11,3,6,13,10]","[0,4,2,7,6,3,13,10,1,5,8,15,12,11,9,17,16,14]"
9,10,WYBNGV,-1,None,None,"[3,5,17,10,13,2,6,1,16,14,15,12,4,11,0,8,9,7]","[14,2,15,13,1,12,17,4,0,9,3,7,6,8,11,10,16,5]"


In [5]:
df_user_study = pd.read_csv(USER_STUDY_CSV)

## Fill Database
If `FILL_DATABASE` is True

In [6]:
%%writefile run_user.py

def run_user(idx, user, url, df_user_study):
    n_learn = 16
    n_eval = 16
    n_users = 10

    mu_got_it_right_pre=0.5
    sigma_got_it_right_pre=0.05
    mu_gain = 0.2
    sigma_gain = 0.1

    def guess(detector_label,p):
        return detector_label if bool(np.random.choice([0,1],p=[1-p, p])) else not detector_label

    import requests
    import json
    import numpy as np
    import pandas as pd
    user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
    user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
  
    res = requests.get(url+"/auth/"+ user["access_token"])

    print(res.text)
    auth_token = json.loads(res.text)
    headers = {'Content-Type': 'application/json','Authorization': "Bearer "+auth_token, "Content-Type": "application/json",}

    requests.post(url+"/api/submitParticipantInfo", json={
    "has_seen_explanation_methods_before": "yes",
    "has_seen_OTHERS_before": "yes",
    "level_of_expertise": "is-researcher-explainability",
    "familiarity_with_chatgpt": "occasional-use",
    "prefers_monochromatic_methods": "yes" if user["access_token"] == "DDEBUG" else "no"
    }, headers=headers)
    # go to phase 2
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 0}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 1}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 2}, headers=headers)

    res = requests.get(url+"/api/state", headers=headers)
    state = json.loads(res.text)

    # user_df = pd.read_sql_query("SELECT * FROM users", connection) # update as group is assigned now
    # user = user_df.iloc[idx]
   # print(user[["detector", "explainer"]])
    # return state
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[state["detector"]],:].reset_index(drop=True)
    for doc_nr, row in df_user_documents.iterrows():
        p_without = user_dist_without()
        requests.post(url+"/api/submitPhase2", json={"ID": doc_nr, "label": guess(row["f(b)"], p_without)}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 3}, headers=headers)

    for doc_nr, row in df_user_documents.iterrows():
        json_ = {"lickert-q{}-{}".format(question_nr, doc_nr): str(np.random.choice([1,2,3,4,5], p=[0.1,0.2,0.1,0.4,0.2])) for question_nr in range(1,4)}
        json_["document_nr"] = doc_nr
        requests.post(url+"/api/submitPhase3", json=json_, headers=headers)

    requests.post(url+"/api/completeCurrentPhase", json={"expected": 4}, headers=headers)
    for doc_nr, row in df_user_documents.iterrows():
        p_with = np.clip(p_without + user_dist_gain(), 0,1)
        requests.post(url+"/api/submitPhase4", json={"ID": doc_nr, "label": guess(row["f(b)"], p_with)}, headers=headers)
    requests.post(url+"/api/completeCurrentPhase", json={"expected": 5}, headers=headers)


Overwriting run_user.py


In [7]:
from tqdm import tqdm
from multiprocess import Pool
from run_user import run_user


if FILL_DATABASE:
    max_pool = 30
    mock_user_data = [(idx, user, server_url, df_user_study) for idx, user in user_df.iterrows()]

    with Pool(max_pool) as p:
        pool_outputs = list(tqdm(p.starmap(run_user,mock_user_data),total=len(mock_user_data)))    
    print(pool_outputs)


ValueError: probabilities do not sum to 1

In [ ]:
user_df = pd.read_sql_query("SELECT * FROM users", connection) # update df from database

In [ ]:
user_df.groupby(["detector", "explainer"])["ID"].count()

Series([], Name: ID, dtype: int64)

In [ ]:
user_df

,ID,access_token,current_phase,detector,explainer,document_order_a,document_order_b


## Evaluation

In [ ]:
def user_metrics(df_user_responses, df_user_study):
    detector = df_user_responses.iloc[0]["detector"]
    # explainer = df_user_responses.iloc[0]["explainer"]
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[detector],:].reset_index(drop=True)
    detector_predictions = df_user_documents["f(b)"].astype(bool)

    user_responses = df_user_responses.loc[df_user_responses.groupby("document_nr")["timestamp"].idxmax()].set_index("document_nr")["label"].astype(bool) # only keep most recent response
    TP = ((detector_predictions) & (user_responses)).sum()
    FP = ((~detector_predictions) & (user_responses)).sum()

    TN = ((~detector_predictions) & (~user_responses)).sum()
    FN = ((detector_predictions) & (~user_responses)).sum()

    acc = (TP+TN) / (TP+FP+TN+FN)
    print("acc", acc)
    print("TP", TP)
    print("FP", FP)
    print("TN", TN)
    print("FN", FN)


    assert sum([TP,FP,TN,FN]) == len(detector_predictions), "Check that input is bool"
    assert (acc ==(user_responses == detector_predictions).sum() / len(detector_predictions)), "Check that input is bool: acc"

    return pd.DataFrame([(TP,TN,FP,FN, acc)], columns=["TP","TN","FP","FN", "User Accuracy"])

In [ ]:
u = user_df.set_index("ID").rename_axis("user_id")[["explainer", "detector"]]


In [ ]:
df_phase_2 = pd.read_sql_query("SELECT responses_phase_2.*, users.detector, users.explainer FROM responses_phase_2 INNER JOIN users ON responses_phase_2.user_id = users.ID", connection)
df_phase_4 = pd.read_sql_query("SELECT responses_phase_4.*, users.detector, users.explainer FROM responses_phase_4 INNER JOIN users ON responses_phase_4.user_id = users.ID", connection)

metrics_phase_4 = df_phase_4.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))
metrics_phase_2 = df_phase_2.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))


difference = metrics_phase_4 - metrics_phase_2 


In [ ]:
difference.join(u).groupby(["detector"])["User Accuracy"].mean()

ValueError: columns overlap but no suffix specified: Index(['detector', 'explainer'], dtype='object')

In [ ]:
difference.join(u).groupby(["explainer"])["User Accuracy"].mean()

explainer
SHAP_Explainer    0.0
Name: User Accuracy, dtype: float64

In [ ]:
difference.join(u).groupby(["detector", "explainer"])["User Accuracy"].mean()

detector           explainer     
DetectorDetectGPT  SHAP_Explainer    0.0
Name: User Accuracy, dtype: float64

In [ ]:
metrics_phase_4.join(u).index.get_level_values(0)

Index([1], dtype='int64', name='user_id')

In [ ]:
def highlight_significant(row, props=''):
  #  display(s)
    styles = [''] * len(row)
    styles[2] = 'font-weight: bold' if row["p value"] <= 0.05 else ''
    return styles

In [ ]:
latex_output = []

In [ ]:
def get_aggregate_results(groupby, label, caption):
    tvalues = []
    pvalues = []
    for name, group_2 in metrics_phase_2.join(u).groupby(groupby):
        group_4 =  metrics_phase_4.join(u)[np.all(metrics_phase_4.join(u)[groupby].values == name, axis=1)]
        tvalue, pvalue = ttest_ind(group_2["User Accuracy"],group_4["User Accuracy"])
        tvalues.append(tvalue)
        pvalues.append(pvalue)

    df_aggregate_results = pd.DataFrame(difference.join(u).groupby(groupby)["User Accuracy"].mean())

    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_2.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"Before"}, axis=1))
    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_4.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"After"}, axis=1))

    df_aggregate_results["t value"] = tvalues
    df_aggregate_results["p value"] = pvalues

    df_aggregate_results.rename(columns={"User Accuracy":"Increase in User Accuracy"}, inplace=True)
    df_aggregate_results = df_aggregate_results.reindex(sorted(df_aggregate_results.columns), axis=1)
    result = df_aggregate_results.style.apply(highlight_significant, axis=1)\
        .map_index(lambda v: "rotatebox:{45}--rwrap;", level=0, axis=1).format(precision=2).hide(["t value"], axis=1).format_index(escape="latex", axis=0)
    latex_output.append(result.to_latex(environment="longtable", 
                                        convert_css=True, 
                                        clines="all;data", 
                                        hrules=True, 
                                        caption=caption, 
                                        label=label))
    return result

In [ ]:
get_aggregate_results(["detector"], "resultsuserstudydetector", "Results aggregated by detector")

c:\Users\loris\thesis\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7030: RuntimeWarning: invalid value encountered in scalar divide
  svar = ((n1 - 1) * v1 + (n2 - 1) * v2) / df


,After,Before,Increase in User Accuracy,p value
detector,,,,
DetectorDetectGPT,0.89,0.89,0.00,nan


In [ ]:
get_aggregate_results(["explainer"], "resultsuserstudyexplainer", "Results aggregated by explainer")

c:\Users\loris\thesis\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7030: RuntimeWarning: invalid value encountered in scalar divide
  svar = ((n1 - 1) * v1 + (n2 - 1) * v2) / df


,After,Before,Increase in User Accuracy,p value
explainer,,,,
SHAP\_Explainer,0.89,0.89,0.00,nan


In [ ]:
get_aggregate_results(["explainer", "detector"], "resultsuserstudyexplainerdetector", "Results aggregated by explainer and detector pairing")

c:\Users\loris\thesis\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7030: RuntimeWarning: invalid value encountered in scalar divide
  svar = ((n1 - 1) * v1 + (n2 - 1) * v2) / df


,,After,Before,Increase in User Accuracy,p value
explainer,detector,,,,
SHAP\_Explainer,DetectorDetectGPT,0.89,0.89,0.00,nan


## Lickert Items

In [ ]:
df_phase_3 = pd.read_sql_query("SELECT responses_phase_3.*, users.detector, users.explainer FROM responses_phase_3 INNER JOIN users ON responses_phase_3.user_id = users.ID", connection)
df_phase_3

,ID,timestamp,label,user_id,document_nr,question_nr,detector,explainer
0,1,2024-03-02 23:13:59,2,1,12,1,DetectorDetectGPT,SHAP_Explainer
1,2,2024-03-02 23:14:13,2,1,12,1,DetectorDetectGPT,SHAP_Explainer
2,3,2024-03-02 23:14:13,4,1,12,2,DetectorDetectGPT,SHAP_Explainer
3,4,2024-03-02 23:14:47,2,1,12,1,DetectorDetectGPT,SHAP_Explainer
4,5,2024-03-02 23:14:47,4,1,12,3,DetectorDetectGPT,SHAP_Explainer
...,...,...,...,...,...,...,...,...
280,281,2024-03-02 23:55:19,4,1,10,1,DetectorDetectGPT,SHAP_Explainer
281,282,2024-03-02 23:55:19,2,1,10,5,DetectorDetectGPT,SHAP_Explainer
282,283,2024-03-02 23:55:19,4,1,10,3,DetectorDetectGPT,SHAP_Explainer
283,284,2024-03-02 23:55:19,3,1,10,4,DetectorDetectGPT,SHAP_Explainer


In [ ]:
user_responses = df_phase_3.loc[df_phase_3.groupby(["user_id", "question_nr", "document_nr"])["timestamp"].idxmax()].set_index(["user_id", "document_nr", "question_nr"]).drop(["timestamp", "ID"], axis=1)
user_responses

label           detector       explainer
user_id document_nr question_nr                                          
1       0           1                3  DetectorDetectGPT  SHAP_Explainer
        1           1                4  DetectorDetectGPT  SHAP_Explainer
        2           1                2  DetectorDetectGPT  SHAP_Explainer
        3           1                4  DetectorDetectGPT  SHAP_Explainer
        4           1                3  DetectorDetectGPT  SHAP_Explainer
...                                ...                ...             ...
        13          5                2  DetectorDetectGPT  SHAP_Explainer
        14          5                2  DetectorDetectGPT  SHAP_Explainer
        15          5                3  DetectorDetectGPT  SHAP_Explainer
        16          5                2  DetectorDetectGPT  SHAP_Explainer
        17          5                3  DetectorDetectGPT  SHAP_Explainer

[90 rows x 3 columns]

In [ ]:
user_responses.groupby(["detector", "explainer", "question_nr"]).mean()

label
detector          explainer      question_nr          
DetectorDetectGPT SHAP_Explainer 1            3.277778
                                 2            3.388889
                                 3            3.222222
                                 4            3.277778
                                 5            2.555556

In [ ]:
groupby = [ "explainer", "document_nr"]
alpha_values = []
for name, group in user_responses.groupby(groupby):
    cannonical_form = group.reset_index().pivot(columns=["question_nr"], values=["label"], index=["user_id", "document_nr"])
    # print(cannonical_form)
    # print()
    alpha_values.append((*name, krippendorff.alpha(cannonical_form, level_of_measurement="ordinal")))
df_krippendorff_alpha = pd.DataFrame(alpha_values, columns=groupby + ["alpha"])
df_krippendorff_alpha.groupby(groupby[0:-1]).describe()["alpha"]


ValueError: There has to be at least one unit with values assigned by at least two coders.

In [ ]:
def highlight_significant(row, props=''):
  #  display(s)
    styles = [''] * len(row)
    styles[0] = 'font-weight: bold' if row["p value"] <= 0.05 else ''
    return styles

In [ ]:
def get_aggregate_results_lickert(groupby, label, caption):
    tvalues = []
    pvalues = []
    for name, group in user_responses.groupby(groupby):
        tvalue, pvalue = ttest_1samp(group["label"], popmean=3)
        tvalues.append(tvalue)
        pvalues.append(pvalue)
        
        

    df_aggregate_results = pd.DataFrame(user_responses.groupby(groupby)["label"].mean())


    df_aggregate_results["t value"] = tvalues
    df_aggregate_results["p value"] = pvalues

    df_aggregate_results = df_aggregate_results.reindex(sorted(df_aggregate_results.columns), axis=1)
    result = df_aggregate_results.style.apply(highlight_significant, axis=1)\
        .map_index(lambda v: "rotatebox:{45}--rwrap;", level=0, axis=1).format(precision=2).hide(["t value"], axis=1).format_index(escape="latex", axis=0)
    latex_output.append(result.to_latex(environment="longtable", 
                                        convert_css=True, 
                                        clines="all;data", 
                                        hrules=True, 
                                        caption=caption, 
                                        label=label))

    
    return result


In [ ]:
get_aggregate_results_lickert(["detector", "explainer", "question_nr"], "lickert-detector-explainer","Lickert Scale Items on detector-explainer level")

In [ ]:
get_aggregate_results_lickert([ "explainer", "question_nr"], "lickert-explainer","Lickert Scale Items on explainer level")

In [ ]:
for  l in latex_output:
    print(l)

\begin{longtable}{lrrrr}
\caption{Results aggregated by detector} \label{resultsuserstudydetector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endfirsthead
\caption[]{Results aggregated by detector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endhead
\midrule
\multicolumn{5}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
DetectorGuo & 0.75 & 0.67 & 0.08 & nan \\
\cline{1-5}
DetectorRadford & 0.92 & 0.75 & 0.17 & nan \\
\cline{1-5}
\end{longtable}

\begin{longtable}{lrrrr}
\caption{Results aggregated by explainer} \label{resultsuserstudyexplainer} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
explainer &  &  &  &  \\
\midrule
\e

In [ ]:




# # "Reproducing" information from Table 1/2 in Hase et al.:


# user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
# df_change = user_metrics_eval - user_metrics_pre
# df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

# user_acc_col = df_change["Change in User Accuracy"] # for convenience

# # use student t for low number of samples
# lower, upper = stats.t.interval(
# confidence=0.95, 
# df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#             loc=user_acc_col.mean(), 
#             scale=stats.sem(user_acc_col)
#             ) 

# p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

# k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



# lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval

# # print results
# #  print("Mean change in acc",user_acc_col.mean())
# # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))


# #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
# #   print("Krippendorff between users: {}".format(k_alpha))
# return p_val

In [ ]:
# def evaluate_user_study(df_user_study, df_phase_2, df_phase_4):
#     # "Reproducing" information from Table 1/2 in Hase et al.:
#     user_metrics_pre = df_phase_2.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study))

#     user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
#     df_change = user_metrics_eval - user_metrics_pre
#     df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

#     user_acc_col = df_change["Change in User Accuracy"] # for convenience

#     # use student t for low number of samples
#     lower, upper = stats.t.interval(
#     confidence=0.95, 
#     df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#               loc=user_acc_col.mean(), 
#               scale=stats.sem(user_acc_col)
#               ) 
    
#     p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

#     k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



#     lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval
    
#     # print results
#   #  print("Mean change in acc",user_acc_col.mean())
#    # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))

    
#   #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
#  #   print("Krippendorff between users: {}".format(k_alpha))
#     return p_val
    

In [ ]:
# def simulate_hase(

#         n_learn = 16,
#         n_eval = 16,
#         n_users = 10,

#         mu_got_it_right_pre=0.5,
#         sigma_got_it_right_pre=0.05,

#         mu_gain = 0.1,
#         sigma_gain = 0.1,


# ):
#     users = []
#     user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
#     user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
#     for i in range(1, n_users+1):
#         p_without = user_dist_without()
#         p_with = np.clip(p_without + user_dist_gain(), 0,1)
#         users.append(("u_%s" % i, p_without ,p_with))
#     documents_learn_1_2 = ["l_%s" % i for i in range(1,n_learn+1)]
#     documents_pre_eval = ["e_%s" % i for i in range(1,n_eval+1)]

#     df_detector_output = mock_detector_responses(documents_pre_eval)

#     responses_pre, responses_eval = mock_user_responses(df_detector_output, documents_pre_eval, users)
#     df_pre =pd.DataFrame(responses_pre, columns=columns_experiment)
#     df_eval =pd.DataFrame(responses_eval, columns=columns_experiment)

#  #   print("# responses pre", len(responses_pre))
#   #  print("# responses pre per method", len(responses_pre)/3)
#    # print("Each user saw {} instances. ".format(2*n_learn + 2*n_eval) )
#    # print("Used {} unique documents. A set of {} in phase 1 and 3; and a set of {} in phase 2 and 4.".format(n_learn + n_eval,n_learn, n_eval))

# #    print("Results based on {} unique eval documents.".format(n_eval))

# #    print("Results based on {} datapoints.".format(len(responses_eval)))
#     p_value = evaluate_user_study(df_detector_output, df_pre, df_eval)
#     return p_value